In [20]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import random
from xgboost import XGBClassifier

In [52]:
train_df = pd.read_csv("train.csv")
x_train = train_df.drop(["SalePrice"], axis = 1)
y_train = train_df["SalePrice"]
x_test = pd.read_csv('test_toup.csv')

In [53]:
x_train.shape

(1321, 80)

In [54]:
x_train["type"] = "train"
x_test["type"] = "test"

In [55]:
x_whole = pd.concat([x_train, x_test], axis = 0, ignore_index=True)
print(x_train.shape)
print(x_test.shape)
print(x_whole.shape)

(1321, 81)
(139, 81)
(1460, 81)


In [56]:
object_dtype = []

for i in x_whole.columns:
    if x_whole[i].dtype == 'object':
        object_dtype.append(i)
        
np.array(object_dtype)

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition', 'type'], dtype='<U13')

In [57]:
x_whole['MasVnrType'].unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn', nan], dtype=object)

In [58]:
valid_na_columns = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [59]:
num_but_obj_columns = ['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd','GarageCars', 'Fireplaces', 'GarageYrBlt', 'MoSold', 'YrSold']

In [60]:
#for int and float columns
meanable_cols = []


for i in x_whole.columns:
    if i not in num_but_obj_columns:
        if x_whole[i].dtypes == 'int' or x_whole[i].dtypes == 'float':
            meanable_cols.append(i)
            
meanable_cols

['Id',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [63]:
le = preprocessing.LabelEncoder()

for i in x_whole.columns:
    if i not in valid_na_columns:
        if i in meanable_cols:
            x_whole[i].fillna(-1, inplace = True)
            for j in range(len(train_df[i])):
                if train_df[i][j] < 0:
                    x_whole[i][j] = random.choice([x_whole[i].mean() - x_whole[i].std(), x_whole[i].mean() + x_whole[i].std()])
        else:
            print(i)
            x_whole[i].fillna("None", inplace = True)
            for j in range(len(x_whole[i])):
                if x_whole[i][j] == "None":
                    x_whole[i][j] = random.choice(x_whole[i].dropna().unique())
            if i in object_dtype:
                x_whole[i] = le.fit_transform(x_whole[i])
                
    else:
        print(i)
        x_whole[i].fillna('NaN', inplace=True)
        x_whole[i] = le.fit_transform(x_whole[i])

MSSubClass
MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Functional
Fireplaces
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageCars
GarageQual
GarageCond
PavedDrive


/home/rohan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


PoolQC
Fence
MiscFeature
MoSold
YrSold
SaleType
SaleCondition
type


In [64]:
x_whole['GarageYrBlt'] = x_whole['GarageYrBlt'].astype(float)
x_whole['GarageYrBlt'].unique()

array([1997., 1989., 2005., 2000., 1999., 1963., 2004., 1931., 2001.,
       1946., 1936., 1975., 2007., 1971., 1960., 1923., 2009., 1950.,
       2006., 1984., 2003., 1925., 1926., 1986., 1955., 1981., 1991.,
       1987., 1961., 1988., 1958., 1935., 1920., 1957., 1993., 1990.,
       1916., 1979., 1932., 1972., 1976., 1918., 1929., 1980., 1924.,
       1948., 1962., 1964., 1996., 1977., 1940., 1939., 2002., 1995.,
       1974., 1959., 1949., 2008., 1994., 1910., 1954., 1978., 1956.,
       1965., 1966., 1982., 1951., 1992., 1928., 1970., 1953., 1968.,
       1998., 1983., 1941., 1930., 2010., 1915., 1985., 1927., 1947.,
       1933., 1937., 1973., 1942., 1938., 1967., 1952., 1922., 1934.,
       1969., 1900., 1906., 1921., 1914., 1945., 1908.])

In [66]:
le.inverse_transform(x_whole["type"])

array([1, 1, 1, ..., 0, 0, 0])

In [67]:
x_train = x_whole[x_whole["type"] == 1]
x_test = x_whole[x_whole["type"] == 0]
print(x_train.shape)
print(x_test.shape)
x_train.drop("type", axis = 1, inplace=True)
x_test.drop("type", axis = 1, inplace = True)

(1321, 81)
(139, 81)


/home/rohan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [68]:
model = XGBClassifier()

In [69]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [70]:
y_pred = model.predict(x_test)

In [71]:
y_pred

array([235000, 175000, 205000, 122000, 280000, 154000, 207500, 236500,
       115000, 141000, 125000, 239000, 137000, 255500, 148000, 231500,
       148000, 123000, 155000, 127000, 266000, 120000, 191000, 156000,
       115000, 280000, 129000, 250000, 190000,  86000, 119500, 142000,
       266000, 167000, 266000, 225000, 178000, 139000, 132000,  89500,
       139000, 141000, 137000, 156000, 155000, 266000, 184000, 270000,
       135000, 122000, 185000, 127500, 129000, 235000, 115000, 167000,
       172500, 190000, 266000, 127000, 148000,  79000, 266000, 205000,
       237000, 266000, 277000, 204000, 115000, 268000, 260000, 124500,
       190000, 148000, 128000,  89500, 141000, 131000, 125000, 125000,
       185000, 162000, 147000, 148000,  89500, 225000, 212000, 164700,
        67000, 141000,  93500, 140000, 147000, 145000, 231500, 178000,
       204000, 145000,  79000, 132000, 197500, 187000, 125000, 239000,
       118000, 275500, 135000, 110000,  87000, 149500, 145000,  89500,
      

In [72]:
df_out = pd.DataFrame(y_pred)

In [73]:
df_out


,0
0,235000
1,175000
2,205000
3,122000
4,280000
...,...
134,241500
135,152000
136,165000
137,241500


In [74]:
df_out.columns = ["SalePrice"]

In [75]:
df_out["Id"] = df_out.index

In [76]:
df_out = df_out[["Id", "SalePrice"]]

In [77]:
df_out


,Id,SalePrice
0,0,235000
1,1,175000
2,2,205000
3,3,122000
4,4,280000
...,...,...
134,134,241500
135,135,152000
136,136,165000
137,137,241500


In [78]:
df_out["Id"] = range(1, 140)

In [79]:
df_out.set_index("Id")

,SalePrice
Id,
1,235000
2,175000
3,205000
4,122000
5,280000
...,...
135,241500
136,152000
137,165000


In [80]:
df_out.to_csv("ouput.csv", index = False)